### 🔷 One-Hot Encoding

One-Hot Encoding is a technique used to convert **nominal categorical features** into a numerical format.

It creates **binary columns** for each category of a feature — assigning `1` if the value is present, `0` otherwise.

#### ✅ When to Use:
- For **nominal data** (no inherent order), such as `Color = [Red, Green, Blue]`.
- Works well with most machine learning algorithms that don't assume any ordinal relationship between features.

#### 📊 Example:

| Color   | Red | Green | Blue |
|---------|-----|-------|------|
| Red     | 1   | 0     | 0    |
| Green   | 0   | 1     | 0    |
| Blue    | 0   | 0     | 1    |

#### 🔁 Considerations:
- **High cardinality** (many unique categories) can lead to **dimensionality explosion** (many columns).
- Leads to **sparse data**, which some algorithms may not handle well without optimization.

#### ⚠️ Dummy Variable Trap:
- If all `n` binary columns are included, they can become **linearly dependent**.
- To avoid this, we usually **drop one column** (e.g., use only `n - 1` columns).

#### 🔍 When NOT to Use:
- For **ordinal data** (use Ordinal Encoding instead).
- When category frequency is too high — consider techniques like frequency/target encoding or dimensionality reduction.

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("cars.csv")
df.head()

,brand,km_driven,fuel,owner,selling_price
0,Maruti,145500,Diesel,First Owner,450000
1,Skoda,120000,Diesel,Second Owner,370000
2,Honda,140000,Petrol,Third Owner,158000
3,Hyundai,127000,Diesel,First Owner,225000
4,Maruti,120000,Petrol,First Owner,130000


In [7]:
df['owner'].value_counts()

owner
First Owner             5289
Second Owner            2105
Third Owner              555
Fourth & Above Owner     174
Test Drive Car             5
Name: count, dtype: int64

### 1️⃣ OneHotEncoding using Pandas

In [11]:
pd.get_dummies(df, columns=['fuel', 'owner'], dtype=int).head()

,brand,km_driven,selling_price,fuel_CNG,fuel_Diesel,fuel_LPG,fuel_Petrol,owner_First Owner,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,Maruti,145500,450000,0,1,0,0,1,0,0,0,0
1,Skoda,120000,370000,0,1,0,0,0,0,1,0,0
2,Honda,140000,158000,0,0,0,1,0,0,0,0,1
3,Hyundai,127000,225000,0,1,0,0,1,0,0,0,0
4,Maruti,120000,130000,0,0,0,1,1,0,0,0,0


### 2️⃣ K-1 OneHotEncoding

In [12]:
pd.get_dummies(df, columns=['owner', 'fuel'], drop_first=True, dtype=int).head()

,brand,km_driven,selling_price,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner,fuel_Diesel,fuel_LPG,fuel_Petrol
0,Maruti,145500,450000,0,0,0,0,1,0,0
1,Skoda,120000,370000,0,1,0,0,1,0,0
2,Honda,140000,158000,0,0,0,1,0,0,1
3,Hyundai,127000,225000,0,0,0,0,1,0,0
4,Maruti,120000,130000,0,0,0,0,0,0,1


### 3️⃣ OneHotEncoding using Sklearn

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :4], df.iloc[:, :-1], test_size=0.2, random_state=0)

In [14]:
X_train.head()

,brand,km_driven,fuel,owner
3042,Hyundai,60000,LPG,First Owner
1520,Tata,150000,Diesel,Third Owner
2611,Hyundai,110000,Diesel,Second Owner
3544,Mahindra,28000,Diesel,Second Owner
4138,Maruti,15000,Petrol,First Owner


In [15]:
from sklearn.preprocessing import OneHotEncoder

In [16]:
ohe = OneHotEncoder(drop='first', sparse_output=False, dtype=int)

In [17]:
X_train_new = ohe.fit_transform(X_train[['fuel', 'owner']])
X_test_new = ohe.transform(X_test[['fuel', 'owner']])

In [19]:
X_train_new

array([[0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 1, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]], shape=(6502, 7))

In [20]:
X_test_new

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 0, 0]], shape=(1626, 7))

In [ ]:
# here you can see, we have to concatenate the new features with the original dataframe
# to not do this we will learn how to use ColumnTransformer
df_new = np.hstack((X_train[['brand', 'km_driven']].values, X_train_new))
df_new

array([['Hyundai', 60000, 0, ..., 0, 0, 0],
       ['Tata', 150000, 1, ..., 0, 0, 1],
       ['Hyundai', 110000, 1, ..., 1, 0, 0],
       ...,
       ['Hyundai', 90000, 0, ..., 1, 0, 0],
       ['Volkswagen', 90000, 1, ..., 0, 0, 0],
       ['Hyundai', 110000, 0, ..., 0, 0, 0]],
      shape=(6502, 9), dtype=object)

In [29]:
df_new_new = pd.DataFrame(df_new, columns=['brand', 'km_driven'] + list(ohe.get_feature_names_out(['fuel', 'owner'])))
df_new_new

,brand,km_driven,fuel_Diesel,fuel_LPG,fuel_Petrol,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,Hyundai,60000,0,1,0,0,0,0,0
1,Tata,150000,1,0,0,0,0,0,1
2,Hyundai,110000,1,0,0,0,1,0,0
3,Mahindra,28000,1,0,0,0,1,0,0
4,Maruti,15000,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
6497,Tata,70000,1,0,0,0,0,0,1
6498,Ford,100000,1,0,0,0,1,0,0
6499,Hyundai,90000,0,0,1,0,1,0,0
6500,Volkswagen,90000,1,0,0,0,0,0,0


### 4️⃣ OneHotEncoding with Top Categories

In [30]:
counts = df['brand'].value_counts()
counts

brand
Maruti           2448
Hyundai          1415
Mahindra          772
Tata              734
Toyota            488
Honda             467
Ford              397
Chevrolet         230
Renault           228
Volkswagen        186
BMW               120
Skoda             105
Nissan             81
Jaguar             71
Volvo              67
Datsun             65
Mercedes-Benz      54
Fiat               47
Audi               40
Lexus              34
Jeep               31
Mitsubishi         14
Land                6
Force               6
Isuzu               5
Ambassador          4
Kia                 4
MG                  3
Daewoo              3
Ashok               1
Opel                1
Peugeot             1
Name: count, dtype: int64

In [34]:
df['brand'].nunique()
threshold = 100

In [35]:
repl = counts[counts < threshold].index

In [36]:
pd.get_dummies(df['brand'].replace(repl, 'other'), dtype=int).head()

,BMW,Chevrolet,Ford,Honda,Hyundai,Mahindra,Maruti,Renault,Skoda,Tata,Toyota,Volkswagen,other
0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0
